In [1]:
!git clone https://github.com/AnshuLodha007/Unicorn_Hackathon.git

fatal: destination path 'Unicorn_Hackathon' already exists and is not an empty directory.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pandas import Series
%matplotlib inline

from sklearn.metrics import mean_squared_error
from math import sqrt
from statistics import mean

import warnings
warnings.filterwarnings("ignore")

In [7]:
# Reading the csv files

#train = pd.read_csv('/Unicorn_Hackathon/Train.csv')
#train = pd.read_csv('/Unicorn_Hackathon/Test.csv')

train = pd.read_csv(rf'C:\Dropbox\GenAI\Analytics_Vidhya\Time_Series_Forecasting_using_Python\Train.csv')
test = pd.read_csv(rf'C:\Dropbox\GenAI\Analytics_Vidhya\Time_Series_Forecasting_using_Python\Test.csv')
train.shape, test.shape

((18288, 3), (5112, 2))

In [9]:
# making copy of orignal data.

train_orignal = train.copy()
test_orignal = test.copy()